# Initial class creation

In [3]:
class MyTradingStrategy:

    def __init__(self, name):

        self.__name = name

    def generate_signal(self, price_data):
        print('This method is intended to be overwritten')
        return 'Hold'
    
    @property
    def name(self):
        return self.__name

In [4]:
MyBaseObj = MyTradingStrategy('Awesome Strategy')
MyBaseObj.generate_signal(13)
MyBaseObj.name

This method is intended to be over ridded


'Awesome Strategy'

# SMA trading strategy

In [7]:
class MySMATradingStrategy(MyTradingStrategy):

    def __init__(self, swindow, lwindow):
        self.__swindow = swindow
        self.__lwindow = lwindow
        super().__init__('MY SMA Trading strategy')

    def generate_signal(self, price_data):

        if len(price_data[-self.__lwindow:]) < self.__lwindow:
            return 'Hold'
        
        short_avg = sum(price_data[-self.__swindow:]) / self.__swindow
        long_avg = sum(price_data[-self.__lwindow:]) / self.__lwindow

        if short_avg > long_avg:
            return 'Buy'
        elif short_avg < long_avg:
            return 'Sell'
        else:
            return 'Hold'

    @property
    def swindow(self):
        return self.__swindow
    
    @property
    def lwindow(self):
        return self.__lwindow

In [9]:
ObjStrategy = MySMATradingStrategy(3, 5)
print(ObjStrategy.generate_signal([12, 14, 15, 14, 2, 3, 8, 9, 7, 14]))
print(ObjStrategy.lwindow)
print(ObjStrategy.swindow)
print(ObjStrategy.name)

Buy
5
3
MY SMA Trading strategy


# MyTrade class creation

In [10]:
from datetime import datetime 

In [20]:
class MyTrade:

    def __init__(self, strategy_name, signal, amount):
        self.__strategy_name = strategy_name
        self.__signal = signal
        self.__amount = amount
        self.__timestamp = datetime.now()

    def execute(self):
        print(f'Executed {self.__signal} trade with the strategy <<{self.__strategy_name}>> for amount of {self.__amount} at {self.__timestamp}')

    @property
    def signal(self):
        return self.__signal
    
    @property
    def strategy_name(self):
        return self.__strategy_name
    
    @property
    def amount(self):
        return self.__amount
    
    @property
    def timestamp(self):
        return self.__timestamp

In [21]:
strategy_name = ObjStrategy.name 
signal = ObjStrategy.generate_signal([12, 14, 15, 14, 2, 3, 8, 9, 7, 14])
print(strategy_name, signal)

ObjMyTrade = MyTrade(strategy_name, signal, 10000)
ObjMyTrade.execute()

MY SMA Trading strategy Buy
Executed Buy trade with the strategy <<MY SMA Trading strategy>> for amount of 10000 at 2024-06-24 21:09:35.423006


# Mock Trading API creation

In [22]:
class MockTradingAPI:

    def __init__(self, balance):
        self.__balance = balance

    def place_order(self, trade, price):
        if trade.signal == 'Buy' and self.__balance >= trade.amount *price:
            self.__balance -= trade.amount * price 
            print(f'Placed a buy trade at {price}, Remaining Balance: {self.__balance}')
        elif trade.signal == 'Sell':
            self.__balance += trade.amount * price 
            print(f'Placed a sell trade at {price}, Remaining Balance: {self.__balance}')
        else:
            print('Insufficient balance or Invalid signal')

    @property
    def balance(self):
        return self.__balance

In [28]:
trade = MyTrade(strategy_name, signal, 10000)
ObjMockAPI = MockTradingAPI(10000)
ObjMockAPI.place_order(trade, 1000)

Insufficient balance or Invalid signal


# Trading system class creation

In [29]:
! pip install yfinance -q

In [41]:
import yfinance as yf
import time

In [42]:
class MyTradingSystem:

    def __init__(self, api, strategy, symbol):
        self.__api = api
        self.__strategy = strategy
        self.__symbol = symbol
        self.__price_data = []

    def fetch_price_data(self):
        data = yf.download(self.__symbol, period='1d', interval='1m')
        if not data.empty:
            price = data['Close'].iloc[-1]
            self.__price_data.append(price)
            if len(self.__price_data) > self.__strategy.lwindow:
                self.__strategy.lwindow.pop(0)
                print(f'Fetched new price data: {price}')
            else:
                print('No data fetched')


    def run(self):
        self.fetch_price_data()
        signal = self.__strategy.generate_signal(self.__price_data)
        print(f'Generated signal: {signal}')
        if signal in ['Sell', 'Buy']:
            trade = MyTrade(self.__strategy.name, signal, 1)
            trade.execute()
            self.__api.place_order(trade, self.__price_data[-1])

    @property
    def api(self):
        return self.__api
    
    @property
    def strategy(self):
        return self.__strategy
    
    @property
    def symbol(self):
        return self.__symbol
    
    @property
    def price_data(self):
        return self.__price_data

In [43]:
api = MockTradingAPI(balance=10000)
symbol = 'AAPL'
strategy = MySMATradingStrategy(swindow=3, lwindow=5)
system = MyTradingSystem(api, strategy, symbol)

for _ in range(3):
    system.run()
    print(f'Remaining balance: {api.balance}')
    time.sleep(2)

[*********************100%%**********************]  1 of 1 completed


No data fetched
Generated signal: Hold
Remaining balance: 10000


[*********************100%%**********************]  1 of 1 completed


No data fetched
Generated signal: Hold
Remaining balance: 10000


[*********************100%%**********************]  1 of 1 completed


No data fetched
Generated signal: Hold
Remaining balance: 10000


# File handling

In [2]:
f = open('trades.csv', 'w')
f.write('This is an algo trading course.')
f.close()

In [3]:
f = open('trades.csv', 'r')
f.read()

'This is an algo trading course.'

In [4]:
f.close()

In [5]:
f = open('trades.txt', 'w')
f.write('I hope this course will be helpful.')
f.close()

In [6]:
temp_list = ['Hi bebe', '\n How are you doing?']
f = open('trades.txt', 'w')
f.writelines(temp_list)
f.close()

In [7]:
f = open('trades.txt', 'r')
f.readlines()

['Hi bebe\n', ' How are you doing?']

In [8]:
f.close()

In [10]:
with open('trades.txt', 'r') as f:
    print(f.readline())

Hi bebe



In [11]:
with open('trades.txt', 'a') as f:
    f.write('I am ok.')

In [12]:
with open('trades.txt', 'r') as f:
    print(f.readlines())

['Hi bebe\n', ' How are you doing?I am ok.']


In [13]:
import csv

with open('trades.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Symbol', 'Price'])

In [14]:
with open('trades.txt', 'r') as f:
    print(f.read(5))
    print(f.tell())
    print(f.read(5))
    print(f.tell())
    print(f.seek(11))
    print(f.tell())

Hi be
5
be
 H
11
11
11


In [15]:
temp_list = [
    {
        'trade_id': 'T12345', 'symbol': 'AAPL', 'price': 100, 'volume': 12345
    },
    {
        'trade_id': 'T12344', 'symbol': 'MSFT', 'price': 110, 'volume': 22345
    }
]

In [16]:
import json

# serialization
with open('trade_log.json', 'w') as f:
    json.dump(temp_list, f)

In [17]:
# deserialization
with open('trade_log.json', 'r') as f:
    print(json.load(f))

[{'trade_id': 'T12345', 'symbol': 'AAPL', 'price': 100, 'volume': 12345}, {'trade_id': 'T12344', 'symbol': 'MSFT', 'price': 110, 'volume': 22345}]


In [18]:
import pickle